<a href="https://colab.research.google.com/github/ranaalisaeed/college-student-qa-bot/blob/main/acap_ss_rag_step4_retrieval_n_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages:
# jq: JSON processor for document loader
# langchain-community: LangChain community tools
# chromadb: a vector database for embeddings [deprecated]
# tiktoken: required by OpenAIEmbeddings for tokenization
# langchain-chroma: Chroma is deprecated in LangChain 0.2.9

%pip install langchain-openai langchain-chroma tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:00

In [ ]:
import os
from pprint import pprint
import textwrap
import json
from google.colab import drive, userdata
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
drive.mount('/content/drive')
persist_dir = '/content/drive/MyDrive/acap-db'
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embeddings)

Mounted at /content/drive


In [ ]:
len(vectorstore.get()['documents'])

240

In [ ]:
retriever = vectorstore.as_retriever(search_type = 'mmr', search_kwargs = {'k': 3, 'lambda_mult': 0.7})

In [ ]:
retrieved_docs = retriever.invoke(question)

In [ ]:
for doc in retrieved_docs:

  data = json.loads(doc.page_content)
  metadata = doc.metadata

  section_title = data.get('section_title')
  section_content = data.get('section_content')
  metadata_str = ' '.join(f'{key}: {value}' for key, value in metadata.items())

  print(
      f"Section Content: {textwrap.fill(section_content, width=120)}\n"
      f"--------------\nSection Title: {section_title}\n"
      f"--------------\nMetadata: {metadata_str}\n"
  )

Section Content: Why do we need a SSAF? The SSAF model allows funding to areas which would not normally be allocated through operational
costs or cannot be funded due to legislation or education guidelines. It allows ACAP to allocate funding to services,
facilities and student-focused initiatives that would otherwise be unsupported or unable to go ahead. How can I benefit
directly from the SSAF? Services provided through the SSAF funding are made available to all ACAP students, and it is up
to each individual student to choose their level of engagement with these services and activities. What is SA-HELP? SA-
HELP is a Commonwealth loan available to eligible students to pay their Student Services and Amenities Fee (SSAF). I’m
using FEE-HELP, do I still need to pay the SSAF? Your Student Services and Amenities Fee (SSAF) is not covered under
FEE-HELP. However, eligible students may apply for SA-HELP to pay for all or part of the SSAF.
--------------
Section Title: SSAF Frequently asked q

In [ ]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the source page it came from.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [ ]:
chat = ChatOpenAI(model_name="gpt-4", temperature=0.7, max_tokens=500)

In [ ]:
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt_template
    | chat
    | StrOutputParser()
)

In [ ]:
# question = "What is SSAF?"
# question = "As a student, what services are available to me?"
# question = "I need assistance, quickly, who do I contact?"
# question = "What is studiosity and why should I use it?" # no content in knowledge base
# question = "What is learning hub and why should I use it?"
question = "How do I contact IT?"

In [ ]:
print(textwrap.fill(chain.invoke(question), width=120))

ACAP students and staff can get IT assistance from the IT team. They can browse the How To Guides for common
troubleshooting queries relating to the Student Portal, Assessments, and other online resources. However, the document
doesn't provide specific contact details for the IT helpdesk. For general inquiries, you may Contact ACAP at 1800 039
139.  Source: ACAP University College Student Support Services and Work Integrated Learning (Placements) pages.
